In [100]:
import pandas as pd 
import numpy as np

In [101]:
film = pd.read_csv('data/datasets/data_marvel.csv')

In [102]:
sales_2008 = pd.read_csv('data/datasets/sales_2008.csv')
sales_2009 = pd.read_csv('data/datasets/sales_2009.csv')
sales_2010 = pd.read_csv('data/datasets/sales_2010.csv')
sales_2011 = pd.read_csv('data/datasets/sales_2011.csv')
sales_2012 = pd.read_csv('data/datasets/sales_2012.csv')
sales_2013 = pd.read_csv('data/datasets/sales_2013.csv')
sales_2014 = pd.read_csv('data/datasets/sales_2014.csv')
sales_2015 = pd.read_csv('data/datasets/sales_2015.csv')
sales_2016 = pd.read_csv('data/datasets/sales_2016.csv')
sales_2017 = pd.read_csv('data/datasets/sales_2017.csv')
sales_2018 = pd.read_csv('data/datasets/sales_2018.csv')
sales_2019 = pd.read_csv('data/datasets/sales_2019.csv')
sales_2020 = pd.read_csv('data/datasets/sales_2020.csv')
sales_2021 = pd.read_csv('data/datasets/sales_2021.csv')

In [118]:
def generate_last_digits(index):
    num_digits = len(str(index))
    return index % (1 ** (num_digits - 5))

In [117]:
def comics_creator(data, date):
    data['Publish Date'] = date
    second_column = data.pop('Publish Date')
    data.insert(1, 'Publish Date', second_column)
    data['Price'] = data['Price'].replace('[\$,]', '', regex=True).astype(float)
    data['Est. units'] = data['Est. units'].replace(',', '', regex=True).astype(float)
    data['Total'] = data['Price'] * data['Est. units']
    data['Comic_ID'] = data.index
    
    data_list = []
    df = pd.DataFrame(columns=['Comic_ID', 'Publish Date', 'Comic-book Title', 'Price', 'Publisher', 'Est. units', 'Total'])
    
    for index, result in data.iterrows():
        local = {}
        local['Comic_ID'] = result['Comic_ID']
        local['Publish Date'] = result['Publish Date']
        local['Comic-book Title'] = result['Comic-book Title']
        local['Price'] = result['Price']
        local['Publisher'] = result['Publisher']
        local['Est. units'] = result['Est. units']
        local['Total'] = result['Total']
        data_list.append(local)
    df = pd.DataFrame(data_list)
    
    df['Publish Date'] = df['Publish Date'].astype(int)
    last_two_digits = str(date[-2:])
    identifier = {'legend': 315, 'date': last_two_digits, 'index': df.index}
    last_digits = generate_last_digits(df.index)
    df['Comic_ID'] = str(identifier['legend']) + str(identifier['date']) + last_digits.astype(str) + identifier['index'].astype(str)
    
    file_name = f'comic_{date}.csv'
    df.to_csv(f'data/comics_by_year/{file_name}', index=False)   
    return df

In [104]:
def characters_creator(df, check_list, date):
    matches = df[df['Comic-book Title'].str.contains('|'.join(check_list))]
    result_dict = {}
    for valor_b in check_list:
        matching_rows = matches[matches['Comic-book Title'].str.contains(valor_b)]
        if not matching_rows.empty:
            result_dict[valor_b] = {'Name': valor_b, 'Total': matching_rows['Total'].sum()} 
    result_df = pd.DataFrame.from_dict(result_dict, orient='index').reset_index()
    result_df.columns = ['Character_Name', 'Character_ID', 'Total']
    
    identifier = []
    check_letter = {'ch': 38}
    identifier.append(check_letter.get('ch',0))
    result_df['Date'] = date
    result_df['Date'] = result_df['Date'].astype(int)
    last_two_digits = result_df['Date'] % 100
    identifier.append(last_two_digits.unique()[0])
    identifier.append(result_df.index)
    result_df['Character_ID'] = result_df.index
    result_df['Character_ID'] = identifier[0] * 10000 + identifier[1] * 100 + identifier[2]
    
    result_df = result_df[['Character_ID', 'Date', 'Character_Name', 'Total']]
    file_name = f'characters_{date}.csv'
    result_df.to_csv(f'data/characters_by_year/{file_name}', index=False)
    return result_df

In [105]:
def film_creator(df):
    identifier = []
    check_letter = {'m': 13}
    identifier.append(check_letter.get('m',0))
    
    df['Release year'] = df['Release date(United States)'].str.extract(r'(\d{4})')
    df['Release year'] = df['Release year'].astype(int)
    df.drop('Release date(United States)', axis=1, inplace=True)
    second_column = df.pop('Release year')
    df.insert(2, 'Release year', second_column)
    last_two_digits = df['Release year'] % 100
    
    identifier.append(last_two_digits.unique()[0])
    identifier.append(df.index)
    
    df['Film_ID'] = df.index
    df['Film_ID'] = identifier[0] * 10000 + identifier[1] * 100 + identifier[2]
    
    cols = list(df.columns)
    cols = ['Film_ID'] + cols[:-1]
    df = df[cols]
    
    df.to_csv(f'data/film.csv', index=False)
    return df

In [106]:
def join_df(dfs, name):
    df = pd.concat(dfs, ignore_index=True)
    file_name = f'{name}.csv'
    df.to_csv(f'data/{file_name}', index=False)

In [107]:
def id_generator(df, check, name_date, id_name):
    identifier = []
    check_letter = {'ch': 38,'co':315, 'd': 4, 'h': 8, 'm': 13}
    identifier.append(check_letter.get(check, 0))
    last_two_digits = df[name_date] % 100
    identifier.append(last_two_digits.unique()[0])
    identifier.append(df.index)
    df[id_name] = identifier[0] * 10000 + identifier[1] * 100 + identifier[2]
    cols = list(df.columns)
    cols = [id_name] + cols[:-1]
    df = df[cols]
    return df

In [108]:
def save_csv(data, name):
    file_name = f'{name}.csv'
    data.to_csv(f'data/{file_name}', index=False)

In [109]:
def add_date(data):
    data['Release year'] = data['Release date(United States)'].str.extract(r'(\d{4})')
    data['Release year'] = data['Release year'].astype(int)
    data.drop('Release date(United States)', axis=1, inplace=True)
    second_column = data.pop('Release year')
    data.insert(2, 'Release year', second_column)
    return data

In [110]:
check_list = [
    'Absorbing Man', 'A.I.M.', 'Anole', 'Apocalypse', 'Archangel', 'Avalanche', 'Bastion', 'Beast',
    'Beyonder', 'Black Panther', 'Black Widow', 'Blob', 'Cable', 'Cannonball', 'Captain America',
    'Cassandra Nova', 'Chamber', 'Colossus', 'Crossbones', 'Cypher', 'Daredevil', 'Dark Phoenix',
    'Deadpool', 'Doctor Strange', 'Domino', 'Dust', 'Emma Frost', 'Exodus', 'Falcon', 'Fantomex',
    'Feral', 'Forge', 'Gambit', 'Gideon', 'Grandmaster', 'Grim Reaper', 'Havok', 'Hela', 'Hellion',
    'Hope Summers', 'Hulk', 'Iceman', 'Iron Man', 'Iron Patriot', 'Jean Grey', 'Jean Grey', 'Jubilee',
    'Juggernaut', 'Juggernaut', 'Kang', 'Karma', 'Korvac', 'Klaw', 'Lady Deathstrike', 'Living Laser',
    'Loki', 'Longshot', 'Luke Cage', 'M.O.D.O.K.', 'Magneto', 'Marrow', 'Mastermind', 'Master Mold',
    'Marauders', 'Marvel Boy', 'Multiple Man', 'Mystique', 'Nightcrawler', 'Omega Red', 'Onslaught',
    'Pixie', 'Polaris', 'Prodigy', 'Professor X', 'Psylocke', 'Red Skull', 'Rhino', 'Rockslide',
    'Rogue', 'Scarlet Witch', 'Sebastian Shaw', 'Selene', 'Shadowcat', 'Shadow King', 'Shatterstar',
    'Silver Samurai', 'Sinister Six', 'Skrulls', 'Sage', 'Spider-Man', 'Storm', 'Strong Guy', 'Sunspot',
    'Surge', 'Taskmaster', 'Thanos', 'The Hand', 'The Hood', 'Thor', 'U-Foes', 'Ultron', 'Vision',
    'Warpath', 'Wasp', 'William Stryker', 'Wolverine', 'Wolverine', 'X-23','X-Men', 'Avengers',
    'Ghost Rider', 'Venom', 'Dark Phoenix', 'Inhumans', 'Guardians of the Galaxy', 'Logan', 'Fantastic Four',
    'Punisher'
]




In [111]:
film = film_creator(film)

In [112]:
df_08 = comics_creator(sales_2008,'2008')
df_09 = comics_creator(sales_2009, '2009')
df_10 = comics_creator(sales_2010, '2010')
df_11 = comics_creator(sales_2011, '2011')
df_12 = comics_creator(sales_2012, '2012')
df_13 = comics_creator(sales_2013, '2013')
df_14 = comics_creator(sales_2014, '2014')
df_15 = comics_creator(sales_2015, '2015')
df_16 = comics_creator(sales_2016, '2016')
df_17 = comics_creator(sales_2017, '2017')
df_18 = comics_creator(sales_2018, '2018')
df_19 = comics_creator(sales_2019, '2019')
df_20 = comics_creator(sales_2020, '2020')
df_21 = comics_creator(sales_2021, '2021')

In [113]:
comic = join_df([
    df_08, df_09, df_10,
    df_11, df_12, df_13,
    df_14, df_15, df_16,
    df_17, df_18, df_19, 
    df_20, df_21], 
    name = 'comics' 
    )

In [114]:
characters_08 = characters_creator(df_08, check_list, '2008')
characters_09 = characters_creator(df_09, check_list, '2009')
characters_10 = characters_creator(df_10, check_list, '2010')
characters_11 = characters_creator(df_11, check_list, '2011')
characters_12 = characters_creator(df_12, check_list, '2012')
characters_13 = characters_creator(df_13, check_list, '2013')
characters_14 = characters_creator(df_14, check_list, '2014')
characters_15 = characters_creator(df_15, check_list, '2015')
characters_16 = characters_creator(df_16, check_list, '2016')
characters_17 = characters_creator(df_17, check_list, '2017')
characters_18 = characters_creator(df_18, check_list, '2018')
characters_19 = characters_creator(df_19, check_list, '2019')
characters_20 = characters_creator(df_20, check_list, '2020')
characters_21 = characters_creator(df_21, check_list, '2021')


In [115]:
join_df(
        [characters_08, characters_09, characters_10, 
        characters_11, characters_12, characters_13,
        characters_14, characters_15, characters_16,
        characters_17, characters_18, characters_19,
        characters_20, characters_21], 
        name='characters'
        )

In [116]:
df_08

,Comic_ID,Publish Date,Comic-book Title,Price,Publisher,Est. units,Total
0,3150800,2008,Hulk,2.99,Marvel,133895.0,400346.05
1,3150801,2008,Amazing Spider-Man,3.99,Marvel,127856.0,510145.44
2,3150802,2008,Captain America,2.99,Marvel,127524.0,381296.76
3,3150803,2008,Astonishing X-Men,2.99,Marvel,107545.0,321559.55
4,3150804,2008,Uncanny X-Men,2.99,Marvel,105435.0,315250.65
...,...,...,...,...,...,...,...
1199,3150801199,2008,X-Men Worlds Apart,3.99,Marvel,5458.0,21777.42
1200,3150801200,2008,Big Hero 6,3.99,Marvel,5327.0,21254.73
1201,3150801201,2008,Wolverine Power Pack,2.99,Marvel,5149.0,15395.51
1202,3150801202,2008,Lords of Avalon Knights of Darkness,3.99,Marvel,4878.0,19463.22
